In [1]:
# add publication date

In [1]:
from Bio import Entrez
from Bio import SeqIO
from time import sleep

import collections
import pandas as pd 
import re
import time



### Loading the species names (5 most patented wihout hydrothermal vents + 5 most patented from hydrothermal vents) 

In [3]:
df = pd.read_excel('../aar5237_datafiles1.xlsx')

In [83]:
top1 = df[df['Included in this study? '] == 'YES'].sort_values(by = 'Number of patented sequences', ascending= False)[:1]

In [28]:
top6_30 = df[df['Included in this study? '] == 'YES'].sort_values(by = 'Number of patented sequences', ascending= False)[5:30]

### Searching the sequences

In [91]:
list(top6_30['Organism'])[:2]

['salmo salar', 'synechococcus sp.']

In [95]:
# this is half finished --> need to create a dictionary of lists in the loop for each organism --> then iterate through this further down 
batch_size = 100000
#count = 500000 # how am I supposed to know this in advance? --> add a function that it should stop once the returned list is empty
start = 0

AN_dict_perorg = {}
empty_list = []

for organism in list(top6_30['Organism'])[:2]:    # only 2 for now
        ANs = []
        print("Going to identify record %i to %i" % (start + 1, start + batch_size))
        Entrez.email = 'paul.dunshirn@univie.ac.at'
        Entrez.api_key = "a8c851f7d98c18d6c5ca6d26162e9157e808"
        handle = Entrez.esearch(db="nuccore", term=str(organism)+"[Primary Organism] AND ddbj_embl_genbank[filter]", idtype="acc", retstart=start,retmax=batch_size)
        record = Entrez.read(handle)
        handle.close()
        print(record['IdList'][:10])
        ANs.extend(record['IdList'])
        
        while record['IdList'] != empty_list: 
            # need to repeat this as long as no empty list is returned (but have to run it once first so that the list is filled with something )
            sleep(1)
            start = start + batch_size
            print("Going to identify record %i to %i" % (start + 1, start + batch_size))
            Entrez.email = 'paul.dunshirn@univie.ac.at'
            Entrez.api_key = "a8c851f7d98c18d6c5ca6d26162e9157e808"
            handle = Entrez.esearch(db="nuccore", term=str(organism)+"[Primary Organism] AND ddbj_embl_genbank[filter]", idtype="acc", retstart=start,retmax=batch_size)
            record = Entrez.read(handle)
            handle.close()
            print(record['IdList'][:10])
            ANs.extend(record['IdList'])
        
        AN_dict_perorg[str(organism)] = ANs
        start = 0
            
        
        #break # take this away if I want all the organisms

Going to identify record 1 to 100000
['CAJNNT000000000.2', 'ON545969.1', 'ON545966.1', 'ON545936.1', 'ON545930.1', 'OW725360.1', 'OW725361.1', 'OW725362.1', 'OW725363.1', 'OW725364.1']
Going to identify record 100001 to 200000
['ET326618.1', 'ET326617.1', 'ET327384.1', 'ET327383.1', 'ET327382.1', 'ET327381.1', 'ET327380.1', 'ET327379.1', 'ET327378.1', 'ET327377.1']
Going to identify record 200001 to 300000
['ET227097.1', 'ET227096.1', 'ET227095.1', 'ET227094.1', 'ET227093.1', 'ET227092.1', 'ET227091.1', 'ET227090.1', 'ET227089.1', 'ET225864.1']
Going to identify record 300001 to 400000
['EG921474.1', 'EG921473.1', 'EG921472.1', 'EG921471.1', 'EG921470.1', 'EG921469.1', 'EG921468.1', 'EG921467.1', 'EG921466.1', 'EG921465.1']
Going to identify record 400001 to 500000
['EG821474.1', 'EG821473.1', 'EG821472.1', 'EG821471.1', 'EG821470.1', 'EG821469.1', 'EG821468.1', 'EG821467.1', 'EG821466.1', 'EG821465.1']
Going to identify record 500001 to 600000
['DY710895.1', 'DY710894.1', 'DY710893.1'

In [98]:
AN_dict_perorg.keys()

dict_keys(['salmo salar', 'synechococcus sp.'])

### Downloading the sequences

Need a csv with AN, origin, pubmedID, authors, publication title, publication date, ... anything else?
--> so I will just append multiple lists with these values
--> but should also have empty values

In [110]:
for k, v in AN_dict_perorg.items():
    print(len(v))

932323
1001


In [132]:
start

0

In [136]:
start = 0
batch_size = 10
for k, v in AN_dict_perorg.items():
    print(v[start:start+batch_size])

['CAJNNT000000000.2', 'ON545969.1', 'ON545966.1', 'ON545936.1', 'ON545930.1', 'OW725360.1', 'OW725361.1', 'OW725362.1', 'OW725363.1', 'OW725364.1']
['ON398846.1', 'BPAC00000000.1', 'BPAC01000001.1', 'BPAC01000002.1', 'BPAC01000003.1', 'BPAC01000004.1', 'BOXD00000000.1', 'BOXD01000001.1', 'BOXD01000002.1', 'BOXD01000003.1']


In [147]:
start_time = time.time() # definitely have to reduce this further 
batch_size = 500000 # this should go up to 400.000, but 500 is recommended by Entrez to not loose anything (?)


data_dict = {}
error_list = []

for k, v in AN_dict_perorg.items():
    try:
        print(k)
        all_pubmed_ids = [] # may want to create one big file that also includes missing values, maybe that would be faster than making to separate lists for pubmedIDs (one with country matches and one without)
        all_sequence_ANs = []
        all_origins = []
        i = 0
        for start in range(0, len(v), batch_size):
            print("Going to download record %i to %i" % (start + 1, start + batch_size))
            Entrez.email = 'paul.dunshirn@univie.ac.at'
            Entrez.api_key = "a8c851f7d98c18d6c5ca6d26162e9157e808"

            handle = Entrez.efetch(db="nuccore", id=v[start:start+batch_size], rettype="gb", retmode="text", idtype="acc") # , retstart=start,retmax=batch_size

            for gb_record in SeqIO.parse(handle, 'genbank'):  

                country = '' # this is not really needed, only in case I want to show countries and non-countries                
                if 'country' in gb_record.features[0].qualifiers:
                    country = re.findall("^(?:(?!:)(?!def).)*",str(gb_record.features[0].qualifiers['country'][0]))[0]

                all_origins.append(country)

                all_sequence_ANs.append(gb_record.name)

                medID = '' 
                if gb_record.annotations['references'][0].pubmed_id: 
                    medID = gb_record.annotations['references'][0].pubmed_id
                all_pubmed_ids.append(medID)
            i = i+1
            if i > 1: 
                break

        data_dict[k] = {'ANs':all_sequence_ANs, 'pubmed':all_pubmed_ids, 'origins':all_origins}
    except:
        print('errorrrrrrrrrrrrr')
        error_list.append(k)
        
    
print("--- %s seconds ---" % (time.time() - start_time))

salmo salar
Going to download record 1 to 500000
errorrrrrrrrrrrrr
synechococcus sp.
Going to download record 1 to 500000
--- 128.33128929138184 seconds ---


In [ ]:
# --> then I have an error list for which I can do it again

In [149]:
data_dict.keys()

dict_keys(['synechococcus sp.'])

In [29]:
# need to adjust this still to export as Json
dictdata = {'origin': all_origins, 'pubMed': all_pubmed_ids, 'ANs': all_sequence_ANs} # maybe I have to write it as dict instead of dictdata? 
df = pd.DataFrame(dictdata)
df.to_csv('C:/Users/pauld/Documents/GitHub/Genetic sequence flows/Data/Genetic sequence data/Entrezdata.csv',index=False)


In [28]:
df

,origin,pubMed,ANs
0,,,OF274803
1,,,OF274776
2,,,OF274680
3,,,OF273781
4,,,OF273744
...,...,...,...
49995,,,LU584487
49996,,,LU584486
49997,,,LU584485
49998,,,LU584484


## Add the looking up samples code here to see whether I can find out anything else? 

## Possibly add a search in open-access papers here? 

## Trash

In [ ]:
# this is to download a limited list of sequence data
Entrez.email = "paul.dunshirn@univie.ac.at"
Entrez.api_key = "a8c851f7d98c18d6c5ca6d26162e9157e808"

all_origins = []
all_sequence_ANss = []

with Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id=takifugu_ids[:50], idtype="acc") as handle:
    for gb_record in SeqIO.parse(handle, "gb"):
            country = '' # this is not really needed, only in case I want to show countries and non-countries                
            if 'country' in gb_record.features[0].qualifiers:
                country = re.findall("^(?:(?!:)(?!def).)*",str(gb_record.features[0].qualifiers['country'][0]))[0]
            
            all_origins.append(country)
            
            all_sequence_ANss.append(gb_record.name)

In [36]:
# this is to download 1 example of sequence data
handle = Entrez.efetch(db="nuccore", id='ON545966.1',rettype="gb", retmode="text") #rettype="gb",
Entrez.email = 'paul.dunshirn@univie.ac.at'
record = SeqIO.read(handle, "genbank")
handle.close()
print(record.annotations['references'][0].pubmed_id)
print(record)


ID: ON545966.1
Name: ON545966
Description: Salmo salar isolate Food_Kit_N_Z cytochrome c oxidase subunit I (COX1) gene, partial cds; mitochondrial
Number of features: 3
/molecule_type=DNA
/topology=linear
/data_file_division=VRT
/date=24-MAY-2022
/accessions=['ON545966']
/sequence_version=1
/keywords=['']
/source=mitochondrion Salmo salar (Atlantic salmon)
/organism=Salmo salar
/taxonomy=['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Actinopterygii', 'Neopterygii', 'Teleostei', 'Protacanthopterygii', 'Salmoniformes', 'Salmonidae', 'Salmoninae', 'Salmo']
/references=[Reference(title='Direct Submission', ...)]
/structured_comment=OrderedDict([('Assembly-Data', OrderedDict([('Sequencing Technology', 'Sanger dideoxy sequencing')]))])
Seq('GGCACCCTCTATTTAGTATTTGGTGCCTGAGCCGGAATAGTCGGCACCGCCCTA...CTG')
